Rewritten NNDescent algorithm using matrices instead of dictionaries.

TODO: Remove for-loops where possible.

In [1]:
!pip install pykeops[colab] > install.log

In [6]:
import torch
import time
import matplotlib.pyplot  as plt
from pykeops.torch import LazyTensor
from pykeops.torch.cluster import cluster_ranges_centroids, from_matrix, sort_clusters
#from collections import defaultdict

use_cuda = torch.cuda.is_available()
if use_cuda:
  torch.cuda.synchronize()
  device = torch.device('cuda')
else:
  device  = torch.device('cpu')

class NNDescent:
  def __init__(self, data, k=2, trees=5, init=4, leaf_multiplier=100, LT=False, a=10):
    # The data is a (N x d) matrix with N instances of d-dimensional points
    self.data = data.to(device)
    N = data.shape[0]
    self.k = k
    self.a = a
    self.numtrees = trees
    self.big_leaves = None
    
    # A 2D tensor representing a directed graph.
    # The value a = graph[i,j] represents an edge from point x_i to x_a.
    self.graph = torch.zeros(size=[N, k], dtype=torch.long)
    
    # Initialize graph randomly or with forest
    if init == 1:
      self.initialize_graph_randomly()
    elif init == 2:
      self.initialize_graph_big_random(data)
    elif init == 3:
      self.initialize_graph_forest(data, leaf_multiplier)
    elif init == 4:
      self.initialize_graph_clusters(data, leaf_multiplier, LT)
    
    # A set of tuples (i,j) of indices for which the distance has already been calculated.
    self.explored_edges = set()

    # A 2D tensor representing the distance between point x_i and x_graph[i,j]
    # self.k_distances = torch.zeros([N, k])
    # self.calculate_all_distances()
    

  def initialize_graph_randomly(self):
    '''
    Initializes self.graph with random values such that each point has k distinct neighbors
    '''
    N, k = self.graph.shape
    # Initialize graph randomly, removing self-loops
    self.graph = torch.randint(high = N-1, size=[N,k], dtype=torch.long)
    row_indices = torch.arange(N).unsqueeze(1).repeat(1,k)
    self.graph[self.graph>=row_indices] += 1

  def initialize_graph_big_random(self, data):
    '''
    Initializes self.graph randomly, but with more neighbours at the start
    '''
    N, k = self.graph.shape
    temp_graph = torch.tensor([])
        
    # make 'trees', combine into giant graph with each element (row) having k * num_trees neighbours
    # this is a small for loop - numtrees and k << datapoints
    for j in range(self.numtrees):
      tree_graph = torch.tensor([])
      for i in range(k):
        tree_graph = torch.cat((tree_graph,torch.randperm(N)),0) # generate randomly shuffled list of N indices
      tree_graph = tree_graph.reshape(-1,k) # creates a N x k tensor with N indices, each appearing k times. This represents 1 'tree'
      temp_graph = torch.cat((temp_graph,tree_graph),1) # combine into giant N x (k*num_trees) tensor. This represents the forest
    
    # find KNN for each row in giant graph
    # TODO - implement the below without a for loop
    for i, row in enumerate(temp_graph):
      temp_row = torch.unique(row).type(torch.LongTensor) # remove duplicates
      temp_row = temp_row[temp_row != i] # remove self
      
      temp_points = data[temp_row,:] # pick out elements from dataset
      distances = dist_bulk(temp_points,data[i]) # Euclidean distances
      indices = distances.topk(k=self.k, largest=False).indices # find indices of KNN
      self.graph[i] = temp_row[indices] # assign KNN to graph
      
  def initialize_graph_forest(self, data, leaf_multiplier):
    '''
    Initializes self.graph with a forest of random trees, such that each point has k distinct neighbors
    '''
    N, k = self.graph.shape
    dim = data.shape[1]
    
    temp_graph = torch.tensor(())
    for j in range(self.numtrees):
      # Create trees, obtain leaves
      t = tree(data, k = k*leaf_multiplier)
      
      # Create temporary graph, 1 for each tree
      # Leaves are of uneven size; select smallest leaf size as graph size
      cols = min([len(leaf) for leaf in t.leaves])
      rows = len(t.leaves)
      tree_graph = torch.zeros((N, cols))
      leaves = torch.tensor(())
      idx_update = torch.tensor(())
      
      # Update graph using leaves
      for leaf in t.leaves:
        temp_idx = torch.as_strided(torch.tensor(leaf).repeat(1,2),size=[len(leaf),cols],stride=[1,1],storage_offset=1)
        tree_graph[leaf,:] = temp_idx.float() # update graph. a lot of overwriting
      # Concatenate all graphs from all trees into 1 giant graph
      temp_graph = torch.cat((temp_graph,tree_graph), 1)

      # Add the first tree's big_leaves to the NNDescent's big_leaves
      if j==0:
          self.big_leaves = torch.LongTensor(t.big_leaves)
    
    warning_count = 0 # number of indices for which some neighbours are random
    # find KNN for each row in giant graph
    # TODO - implement the below without a for loop
    for i, row in enumerate(temp_graph):
      temp_row = torch.unique(row).type(torch.LongTensor) # remove duplicates
      temp_row = temp_row[temp_row != i] # remove self
      
      temp_points = data[temp_row,:] # pick out elements from dataset
      d=((data[i].reshape(1,dim).unsqueeze(1)-temp_points.unsqueeze(0))**2).sum(-1)
      distances, indices = torch.sort(d,dim=1)
      indices = indices.flatten()[:k]
      
      indices = temp_row[indices]
      
      # pad with random indices if there are not enough neighbours
      warning = False # warning flag
      while len(indices) < k:
        pad = torch.randint(high = N-1, size=[k-len(indices),], dtype=torch.long)
        indices = torch.cat((indices,pad))
        indices = torch.unique(indices).type(torch.LongTensor) # remove duplicates
        indices = indices[indices != i] # remove self
        warning = True

      self.graph[i] = indices # assign KNN to graph
      
      if warning:
        warning_count += 1
    
    if warning_count:
      print("WARNING!",warning_count," INDICES ARE RANDOM!")

      
  def initialize_graph_clusters(self, data, leaf_multiplier, LT=False):
    N,dim = data.shape
    k = self.k
    a = self.a
    self.clusters = torch.ones(N,)*-1

    data = data.to(device)
    
    # Create trees, obtain leaves
    t = tree(data, k, leaf_multiplier, LT)
    
    self.leaves = len(t.leaves)

    # Assign each point to a cluster, 1 cluster per tree in forest
    for i, leaf in enumerate(t.leaves):
      self.clusters[leaf] = i
    self.centroids = t.centroids.clone()
    
    # Find nearest centroids
    x_LT=LazyTensor(self.centroids.unsqueeze(1).to(device))
    y_LT=LazyTensor(self.centroids.unsqueeze(0).to(device))
    d=((x_LT-y_LT)**2).sum(-1)
    indices = d.argKmin(K=a+1,dim=1).long()
    self.centroids_neighbours = indices[:,1:].long()

    self.graph = self.centroids_neighbours
    
    # Assign big_leaves by searching for the correct cluster
    self.big_leaves = torch.LongTensor(t.big_leaves)
    for i, index in enumerate(self.big_leaves):
      self.big_leaves[i] = self.clusters[index]
    return
      
  def calculate_all_distances(self):
    '''
    Updates the distances (self.k_distances) of the edges found in self.graph.
    '''
    # Note: Start with for loop for simplicity. TODO: Try to remove loop.
    for i, row in enumerate(self.graph):
      # Indices of current k neighbors in self.graph
      neighbor_indices = [(i,int(r)) for r in row]

      # The distances of those neighbors are saved in k_distances
      self.k_distances[i] = torch.Tensor([dist(self.data[a],self.data[b]) for a,b in neighbor_indices])

      # Add pairs to explored_edges set
      self.explored_edges.update(neighbor_indices) 
    

  def update_graph(self, iter=5):
    '''
      Updates the graph using algorithm: https://pynndescent.readthedocs.io/en/latest/how_pynndescent_works.html
    '''
    # [STEP 1: Start with random graph.] Iterate
    start = time.time()
    for it in range(iter):
#       print("Iteration number",it,"with average distance of",torch.mean(self.k_distances).item(),"Took", time.time()-start,"seconds.")
      has_changed = False

      # [STEP 2: For each node:] (TODO: Investigate whether this can be vectorized.)
      for i, neighbors in enumerate(self.graph):
        # Distances of current neighbors
        dist_current_neighbors = self.k_distances[i]

        # [STEP 3: Measure distance from the node to the neighbors of its neighbors]
        # Find neighbors of neighbors
        potential_neighbors = {a.item() for a in self.graph[neighbors].flatten() \
                               if a not in neighbors and a!=i and (i,int(a)) not in self.explored_edges}
        potential_distances = torch.Tensor([dist(self.data[i],self.data[n]) for n in potential_neighbors])
        self.explored_edges.update([(i,int(r)) for r in potential_neighbors])

        # Concatenate potential neighbors to list of neighbors (indices and distances)
        cat_idx = torch.cat([neighbors, torch.Tensor(list(potential_neighbors))])
        cat_dist = torch.cat([self.k_distances[i], potential_distances])

        # [STEP 4: If any are closer, then update the graph accordingly, and only keep the k closest]
        # Sort using torch.sort(), which also returns sorted indices
        dist_sorted, idx = torch.sort(cat_dist)
        if torch.max(idx[:self.k]) >= self.k:
          has_changed = True
          self.graph[i] = cat_idx[idx[:self.k]]
          self.k_distances[i] = dist_sorted[:self.k]
        
      # [STEP 5: If any changes were made, repeat iteration, otherwise stop]
#       if not has_changed:
#         print("Nothing changed in iteration",it)
#         break
#     print("Done.")

  def k_nearest_graph_search(self,X,max_num_steps = 100, tree_init = True):
    '''
    Gets the k nearest neighbors of input x according to the graph, using this algorithm:
      https://pynndescent.readthedocs.io/en/latest/how_pynndescent_works.html#Searching-using-a-nearest-neighbor-graph
    Input: 
      x - a torch tensor of shape (N,d), where N is the number of dimentions of the input data.
    Output:
      The indices of the k nearest neighbors according using graph search with random initialization.
    '''
    # N datapoints of dimension d
    N, d = X.shape
    X = X.to(device)
    
    k = self.a
    # Boolean mask to keep track of those points whose search is still ongoing
    is_active = (torch.ones(N)==1).to(device)
    
    # If graph was initialized using trees, we can use information from there to initialize in a diversed manner.
    if self.big_leaves is not None and tree_init:
        candidate_idx = self.big_leaves.unsqueeze(0).repeat(N,1).to(device) # Shape: (N,32)
    else:
        # Random initialization for starting points of search. 
        candidate_idx = torch.randint(high=self.graph.shape[0], size=[N, k+1], dtype=torch.long).to(device)
  
    # Sort the candidates by distance from X
    distances = ((self.centroids[candidate_idx] - X.unsqueeze(1))**2).sum(-1)
    sorted, idx = torch.sort(distances, dim=1)
    candidate_idx = torch.gather(candidate_idx, dim=1,index=idx)
    # Truncate to k+1 nearest
    candidate_idx = candidate_idx[:,:(k+1)]
    
    # Track the nodes we have explored already, in N x num_explored tensor
    num_explored = self.k*2
    explored = torch.full(size=[N,num_explored],fill_value=-1).to(device)

    start = time.time()
    # The initialization of candidates and explored set is done. Now we can search.
    count = 0
    while count < max_num_steps:
        # print("Step",count,"- Search is completed for",1-torch.mean(1.0*is_active).item(),"- this step took",time.time()-start,"s")
        start=time.time()
    
        # [2. Look at nodes connected by an edge to the best untried node in graph]
        # diff_bool.shape is (M, k+1, num_explored), where M is the number of active searches
        diff_bool = (candidate_idx[is_active].unsqueeze(2) - explored[is_active].unsqueeze(1) == 0)
        in_explored = torch.any(diff_bool, dim=2)
        # batch_active is true for those who haven't been fully explored in the current batch
        batch_active = ~torch.all(in_explored[:,:-1], dim=1)

        # Update is_active mask. If none are active, break search
        is_active[is_active.clone()] = batch_active
        if not is_active.any():
            break

        # first_unexplored has indices of first unexplored element per row
        first_unexplored = torch.max(~in_explored[batch_active],dim=1)[1].unsqueeze(1) 
        # Unexplored nodes to be expanded
        unexplored_idx = torch.gather(candidate_idx[is_active], dim=1, index=first_unexplored).squeeze(-1)
        explored[is_active,(count%num_explored)] = unexplored_idx
    
        # [3. Add all these nodes to our potential candidate pool]
        # Add neighbors of the first unexplored point to the list of candidates
        expanded_idx = torch.cat((self.graph[unexplored_idx],candidate_idx[is_active]), dim=1) 
    
        # To avoid repeats, we use Hudson's unwanted_indices method to find repeats
        expanded_idx = torch.sort(expanded_idx)[0]
        shift = torch.cat((torch.full((len(expanded_idx),1),-1).to(device),torch.sort(expanded_idx,dim=1)[0][:,:-1]),dim=1)
        unwanted_indices = (expanded_idx==shift)
    
        # [4. Sort by closeness].
        distances = ((self.centroids[expanded_idx] - X[is_active].unsqueeze(1))**2).sum(-1)
        # print(distances.max())
        distances[unwanted_indices] += float('inf')
        sorted, idx = torch.sort(distances,dim=1)
        expanded_idx = torch.gather(expanded_idx,dim=1,index=idx)
    
        # [5. Truncate to k+1 best]
        candidate_idx[is_active] = expanded_idx[:,:(k+1)] 
        
        # [6. Return to step 2. If we have already tried all candidates in pool, we stop in the if not unexplored]
        count += 1
    
    # Return the k candidates
    print("Graph search finished after",count,"steps. Finished for:",1-torch.mean(1.0*is_active).item())
    return candidate_idx[:,:-1]

  def predict(self,x):
    '''
    Predict output using tree. Hasn't been implemented yet. Needs labels y.
    '''
    pass

  def final_brute_force(self, nearest_clusters, query_pts):
    return self._final_brute_force(nearest_clusters,query_pts)

  def _final_brute_force(self, nearest_clusters, query_pts):
    if use_cuda:
      torch.cuda.synchronize()
    k = self.k
    x = self.data.to(device)
    x_labels = self.clusters.long()
    y = query_pts.to(device)
    y_labels = nearest_clusters[:,0]

    x = x.contiguous()
    y = y.contiguous()
    x_labels = x_labels.to(device)
    y_labels = y_labels.to(device)

    clusters, a = self.graph.shape
    r = torch.arange(clusters).repeat(a,1).T.reshape(-1).long()
    keep = torch.zeros([clusters,clusters], dtype=torch.bool).to(device)
    keep[r,self.graph.flatten()] = True
    keep += torch.eye(clusters).bool().to(device)

    x_ranges, x_centroids, _ = cluster_ranges_centroids(x, x_labels)
    y_ranges, y_centroids, _ = cluster_ranges_centroids(y, y_labels)

    x, x_labels = self.__sort_clusters(x, x_labels, store_x=True)
    y, y_labels = self.__sort_clusters(y, y_labels, store_x=False)

    x_LT = LazyTensor(x.unsqueeze(0).to(device).contiguous())
    y_LT = LazyTensor(y.unsqueeze(1).to(device).contiguous())
    D_ij = ((y_LT-x_LT)**2).sum(-1)
    
    x_ranges = x_ranges.to(device)
    y_ranges = y_ranges.to(device)
    ranges_ij = from_matrix(y_ranges, x_ranges, keep)
    D_ij.ranges = ranges_ij
    nn = D_ij.argKmin(K=k,axis=1)
    return self.__unsort(nn)

  def __sort_clusters(self,x,lab,store_x=True):
    lab, perm = torch.sort(lab.view(-1))
    if store_x:
      self.__x_perm=perm 
    else:
      self.__y_perm=perm
    return x[perm],lab

  def __unsort(self,nn):
    return torch.index_select(self.__x_perm[nn],0,self.__y_perm.argsort())
    
    
def dist(x,y):
  # Square of euclidian distance. Skip the root for faster computation.
  return torch.sum((x-y)**2)

def dist_bulk(x,y):
  # Square of euclidian distance. Skip the root for faster computation.
  # For datasets
  return ((x-y)**2).sum(-1)

def dot_product(x, v):
  # Calculate dot product between matrix x and vector v using LazyTensors
  v_LT = LazyTensor(v.view(1,1,-1))
  x_LT = LazyTensor(x.unsqueeze(0))
  return (v_LT|x_LT).sum_reduction(axis=0).flatten()

class tree: # NN clusters tree
  '''
  Random projection tree class that splits the data evenly per split
  Each split is performed by calculating the projection distance of each datapoint to a random unit vector
  The datapoints are then split by the median of of these projection distances
  The indices of the datapoints are stored in tree.leaves, as a nested list
  '''
  def __init__(self, x, k=5, leaf_multiplier=1, LT=False):
    self.min_size = k * leaf_multiplier
    self.leaves = []
    self.sizes = []
    self.centroids = torch.tensor(()).to(device)
    self.big_leaves = [] # leaves at depth = 5
    indices = torch.arange(x.shape[0])

    self.dim = x.shape[1]
    self.data = x.to(device)
    self.LT = LT # Boolean to choose LT or torch initialization

    self.tree = self.make_tree(indices, depth = 0)
    self.centroids = self.centroids.reshape(-1,x.shape[1])

  def make_tree(self, indices, depth):
    if depth == 5: # add to big_leaves if depth=5
      self.big_leaves.append(int(indices[0]))
    if indices.shape[0] > self.min_size:
      v = self.choose_rule().to(device)

      if self.LT:
        distances = dot_product(self.data[indices],v) # create list of projection distances
      else:
        distances = torch.tensordot(self.data[indices],v,dims=1) # create list of projection distances

      median = torch.median(distances)
      left_bool = distances <= median # create boolean array where entries are true if distance <= median
      self.make_tree(indices[left_bool], depth+1)
      self.make_tree(indices[~left_bool], depth+1)
    elif indices.shape[0] != 0:
      self.leaves.append(indices.tolist())
      self.sizes.append(indices.shape[0])
      centroid = self.data[indices].mean(dim=0) # get centroid position
      self.centroids = torch.cat((self.centroids,centroid))
    return

  def choose_rule(self):
    v = torch.rand(self.dim) # create random vector
    v /= torch.norm(v) # normalize to unit vector
    return v
    
def init_accuracy(data, graph, k_distances):
  '''
  Takes in data and graph to check accuracy of graph's assigned k nearest neighbours
  Uses torch brute force to find actual k nearest neighbours
  Returns accuracy: proportion of correct nearest neighbours
  Also returns distance error: (average_distance-true_distances)/true_distance (of k nearest neighbours)
  '''
  N, k = graph.shape

  # Calculate true distances, indices
  d=((data.unsqueeze(1)-data.unsqueeze(0))**2).sum(-1)+torch.Tensor([float('inf')]).repeat(len(data)).diag() # Infinity is added to diagonal
  true_distances, true_indices = torch.sort(d,dim=1)

  # get k nearest neighbours
  true_indices = true_indices[:,:k]
  true_distances = true_distances[:,:k]
  
  # Calculate number of correct nearest neighbours
  accuracy = 0
  for i in range(k):
    accuracy += torch.sum(graph == true_indices).float()
    true_indices = torch.roll(true_indices, 1, -1) # Create a rolling window (index positions may not match)
  accuracy = float(accuracy/(N*k)) # percentage accuracy

  # Calculate accuracy of distances
  true_average = torch.mean(true_distances)
  graph_average = torch.mean(k_distances)
  distance_error = float((graph_average-true_average)/true_average)

  return accuracy, distance_error

def knn_accuracy_x_y(indices_test, x, y):
  '''
  Compares the test and ground truth indices (rows = KNN for each point in dataset)
  Returns accuracy: proportion of correct nearest neighbours
  
  indices_test: k nearest neighbour indices (rows = KNN for each query point)
  x: training points that the model is fitted to
  y: query points for which we want to find the nearest KNN (chosen from x)
  '''
  N, k = indices_test.shape
  
  indices_truth = torch.argsort(((y.unsqueeze(1)-x.unsqueeze(0))**2).sum(-1),dim=1)
  indices_truth = indices_truth[:,:k].to(device)
  
  # Calculate number of correct nearest neighbours
  accuracy = 0
  for i in range(k):
    accuracy += torch.sum(indices_test == indices_truth).float()/N
    indices_truth = torch.roll(indices_truth, 1, -1) # Create a rolling window (index positions may not match)
  accuracy = float(accuracy/k) # percentage accuracy

  return accuracy

def accuracy(indices_test, indices_truth):
  '''
  Compares the test and ground truth indices (rows = KNN for each point in dataset)
  Returns accuracy: proportion of correct nearest neighbours
  '''
  N, k = indices_test.shape
  
  # Calculate number of correct nearest neighbours
  accuracy = 0
  for i in range(k):
    accuracy += torch.sum(indices_test == indices_truth).float()/N
    indices_truth = torch.roll(indices_truth, 1, -1) # Create a rolling window (index positions may not match)
  accuracy = float(accuracy/k) # percentage accuracy

  return accuracy

In [9]:
# Testing out NNDescent class
# torch.manual_seed(1)
data = torch.Tensor([[1.0,1.0], [2.0,1.0], [3.0,1.0], [4.0,1.0],
                     [1.0,2.0], [2.0,2.0], [3.0,2.0], [4.0,2.0]])  
data = torch.randn(size=[1000,4])
# print(data)  

torch.set_printoptions(threshold=10)

k = 5

# Initialize NNDescent graph randomly

print("Initializing RANDOMLY...")
start = time.time()
n = NNDescent(data, k=k, init=1)
print("Took", time.time()-start,"seconds.\n")
print("Graph:")
print(n.graph)
# print("Distances:")
print(torch.sqrt(n.k_distances))

print("Updating...\n")
start = time.time()
n.update_graph(iter=25)
print("Took", time.time()-start,"seconds.\n")
print("Graph:")
print(n.graph)
print("Distances:")
print(torch.sqrt(n.k_distances))
#print(n.k_distances)
accuracy, distance_error = init_accuracy(data, n.graph, n.k_distances)
print("Accuracy: ",accuracy)
print("Distance Error: ",distance_error,'\n')



# Initialize NNDescent graph with big random
print("Initializing BIG RANDOM...")
start = time.time()
n = NNDescent(data, k=k, init=2)
print("Took", time.time()-start,"seconds.\n")
print("Graph:")
print(n.graph)
print("Distances:")
print(torch.sqrt(n.k_distances))

print("Updating...\n")
start = time.time()
n.update_graph(iter=25)
print("Took", time.time()-start,"seconds.\n")
print("Graph:")
print(n.graph)
print("Distances:")
print(torch.sqrt(n.k_distances))
#print(n.k_distances)
accuracy, distance_error = init_accuracy(data, n.graph, n.k_distances)
print("Accuracy: ",accuracy)
print("Distance Error: ",distance_error,'\n')



# Initialize NNDescent graph with forest
print("Initializing FOREST...")
start = time.time()
n = NNDescent(data, k=k, init=3)
print("Took", time.time()-start,"seconds.\n")
print("Graph:")
print(n.graph)
print("Distances:")
print(torch.sqrt(n.k_distances))

print("Updating...\n")
start = time.time()
n.update_graph(iter=25)
print("Took", time.time()-start,"seconds.\n")
print("Graph:")
print(n.graph)
print("Distances:")
print(torch.sqrt(n.k_distances))
#print(n.k_distances)
accuracy, distance_error = init_accuracy(data, n.graph, n.k_distances)
print("Accuracy: ",accuracy)
print("Distance Error: ",distance_error,'\n')



# Brute force search
print("BRUTE FORCE")
start = time.time()
m=((data.unsqueeze(1)-data.unsqueeze(0))**2).sum(-1)#+torch.Tensor([float('inf')]).repeat(len(data)).diag() # Infinity is added to diagonal
distances, brute_force = torch.sort(m,dim=1)
brute_force = brute_force[:,1:k+1]
distances = distances[:,1:k+1]
  
print("Took", time.time()-start,"seconds.\n")
print(brute_force)
print('mean distance',(distances).sqrt().mean())
accuracy, distance_error = init_accuracy(data, brute_force, distances)
print("Accuracy: ",accuracy)
print("Distance Error: ",distance_error,'\n')

# Get k nearest neighbors using graph search
print()
print("Graph search")
#x = torch.Tensor([4, 0]).unsqueeze(0) # size 1 x d
X = torch.randn(size=[1000,data.shape[1]])
print("X is",X)
start = time.time()
k_nearest = n.k_nearest_graph_search(X)
print("Took", time.time()-start,"seconds.")
print("Nearest indices with graph search:\n",k_nearest)

print("K is",k)
# Get k nearest using brute force knn
print("\nActual nearest using KNN")
print("Brute force search:") # Crashes for 1 million points +
start = time.time()
m=((data.unsqueeze(0)-X.unsqueeze(1))**2).sum(-1)
distances, brute_force = torch.topk(m,k=k,dim=1,largest=False)
print("Took", time.time()-start,"seconds.")
print("Nearest indices with brute force:\n",brute_force)


Initializing RANDOMLY...
Took 0.4163076877593994 seconds.

Graph:
tensor([[388, 240, 724, 307, 676],
        [ 39, 410, 547, 393, 341],
        [678, 628, 392, 169, 946],
        ...,
        [327, 449, 480, 136, 973],
        [161, 116, 796,  57, 458],
        [678,  35, 472, 848, 355]])
tensor([[1.8660, 4.4599, 3.3473, 2.3627, 2.2804],
        [1.9366, 2.0393, 2.1813, 1.8742, 1.9006],
        [2.4952, 1.8181, 2.6089, 1.4402, 1.3050],
        ...,
        [2.5797, 2.1167, 1.5458, 1.7441, 2.5093],
        [3.3210, 4.2237, 4.5972, 2.1921, 3.8921],
        [2.0741, 2.4372, 1.8790, 2.4712, 1.6521]])
Updating...



KeyboardInterrupt: ignored

In [11]:
# Comparing the graph search accuracy of tree initialization
N, d = 100000, 2
k = 100 # This k can be different than the graph's k.
a = 5

x = torch.randn(size=[N,d])

start = time.time()
n = NNDescent(x, k=k, init=4, leaf_multiplier=N/32/k, a=a)
print("Fitting took", time.time()-start)

print(n.leaves)

y = torch.randn(size=[10000,x.shape[1]])
#print("X is",X)
print("Search without tree init:")
start = time.time()
k_nearest_random = n.k_nearest_graph_search(y, tree_init = False, max_num_steps=9)
answer_random = n.final_brute_force(k_nearest_random, y)
print("Took", time.time()-start,"seconds.\n")


print("Search with tree init:")
start = time.time()
k_nearest_forest = n.k_nearest_graph_search(y, tree_init = True, max_num_steps=9)
answer_forest = n.final_brute_force(k_nearest_forest, y)
print("Took", time.time()-start,"seconds.")

# print("Nearest indices with graph search:\n",k_nearest)

x_LT=LazyTensor(x.unsqueeze(0).to(device))
y_LT=LazyTensor(y.unsqueeze(1).to(device))
d=((x_LT-y_LT)**2).sum(-1)
brute_force = d.argKmin(K=k,dim=1).long()

# m=((x.unsqueeze(0)-y.unsqueeze(1))**2).sum(-1).to(device)
# distances, brute_force = torch.topk(m,k=k,dim=1,largest=False)

# print(brute_force[0:5])

print("Comparing (naively) the accuracy")
print("Random:",(1.0*(brute_force==answer_random)).mean())
print("Forest:",(1.0*(brute_force==answer_forest)).mean())

print("Accuracy Checkers")
# print('Random:', knn_accuracy_x_y(answer_random,x,y))
# print('Forest:', knn_accuracy_x_y(answer_forest,x,y))
print('Random:', accuracy(answer_random,brute_force))
print('Forest:', accuracy(answer_forest,brute_force))

Fitting took 0.06574106216430664
32
Search without tree init:
Graph search finished after 9 steps. Finished for: 0.9996000000101048
Took 0.30768918991088867 seconds.

Search with tree init:
Graph search finished after 5 steps. Finished for: 1.0
Took 0.24472403526306152 seconds.
Comparing (naively) the accuracy
Random: tensor(0.8963, device='cuda:0')
Forest: tensor(0.8964, device='cuda:0')
Accuracy Checkers
Random: 0.9391242861747742
Forest: 0.9392243027687073
